# REBEL

## Checking REBEL model

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.4 MB/s 
     |████████████████████████████████| 6.6 MB 37.4 MB/s 
     |████████████████████████████████| 163 kB 54.2 MB/s 


In [20]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

# We need to use the tokenizer manually since we need special tokens.
str_data = """Stocks continued to recover from oversold levels, with traders weighing renewed speculation that global central banks could moderate their hawkish stance to prevent a hard landing.
The S&P 500 is having its best two-day surge since November 2020. On top of the obvious short squeeze, a soft economic reading gave bulls a reason for hope when it comes to Federal Reserve policy. 
US job openings sank to a 14-month low -- which may fit well with a central bank that’s very concerned about a hot jobs market. Bond yields fell with the dollar.
Traders also kept a close eye on Tesla Inc., which pared gains substantially on news that Elon Musk is proposing to buy Twitter Inc. for the original offer price of $54.20 a share.
The debate over peak hawkishness has intensified after a dovish surprise from Australia’s central bank and bond buying by the Bank of England.
The idea of a Fed pivot, however, has been met with a lot of skepticism. 
For one, there’s the perception that not much has fundamentally changed to sway officials from their primary goal to knock down inflation. 
Then, there’s the fact that stock pessimism reached such extreme levels that a bounce would be just a matter of when. 
For markets that had been “nearly one-sided,” the liquidation of those positions is a big reason to squeeze in the other direction so vigorously, said Fawad Razaqzada at City Index and Forex.com."""

extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(str_data, return_tensors=True, return_text=False)[0]["generated_token_ids"]])

print(extracted_text[0])

# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)

<s><triplet> Elon Musk <subj> Twitter Inc. <obj> owner of <triplet> Twitter Inc. <subj> Elon Musk <obj> owned by</s>
[{'head': 'Elon Musk', 'type': 'owner of', 'tail': 'Twitter Inc.'}, {'head': 'Twitter Inc.', 'type': 'owned by', 'tail': 'Elon Musk'}]


In [21]:
extracted_triplets

[{'head': 'Elon Musk', 'type': 'owner of', 'tail': 'Twitter Inc.'},
 {'head': 'Twitter Inc.', 'type': 'owned by', 'tail': 'Elon Musk'}]